In [1]:

from analysis.probability import generate_probabilities_simple as probability_generator
from evaluate import evaluate_single

def eval_simple(note):
    return evaluate_single(probability_generator(note), root_note=note)

In [2]:
import numpy as np
from analysis.musical import consonance_ordered_notes, empirical_probabilities
from analysis.spike_tensor import generate_spike_tensor
from cache import get_spikes
from evaluate import predicted_consonance_scores, predicted_probabilities

root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes_spikes = [get_spikes(note) for note in notes]
consonance_ordered_tensors = [generate_spike_tensor(spikes) for spikes in notes_spikes]

root_tensor = generate_spike_tensor(get_spikes(root_note))
probability_tensor = probability_generator(root_note)

def try_eval(tensors, metric, debug=False):
    scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)

    P_predicted = predicted_probabilities(scores)

    if debug:
        def print_arr(arr):
            print(np.round(np.array(arr), 3))
        print("Scores")
        print_arr(scores)
        print("P_predicted")
        print_arr(P_predicted)
        print("P_empirical")
        print_arr(empirical_probabilities())

    print(metric(P_predicted))

In [3]:
from evaluate import js_divergence

try_eval(consonance_ordered_tensors, js_divergence, debug=True)

Scores
[0.119 0.135 0.135 0.136 0.135 0.129 0.132 0.134 0.133 0.124 0.131 0.132]
P_predicted
[0.076 0.086 0.086 0.086 0.086 0.082 0.084 0.085 0.084 0.079 0.083 0.084]
P_empirical
[0.147 0.133 0.12  0.107 0.107 0.093 0.08  0.067 0.053 0.053 0.027 0.013]
0.1912142431741392


In [4]:
import numpy as np

def collect_data(metric, N=10):
    data = []
    for i in range(N):
        A = consonance_ordered_tensors[0][0][0]
        np.random.shuffle(consonance_ordered_tensors)
        B = consonance_ordered_tensors[0][0][0]
        if not np.array_equal(A, B):
            value = try_eval(consonance_ordered_tensors, metric)
            data.append(value)

In [5]:
from evaluate import js_divergence

collect_data(js_divergence)

0.1867308508607451
0.1869592441086574
0.18455535602690212
0.191067307810729
0.19576684114843682
0.1922930766126963
0.19469856689203893
0.19180309122740768


In [6]:
import os

from cache import SPIKES_DIR


root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes = [notes[0], notes[-1]]
notes_spikes = [get_spikes(note) for note in notes]
tensors = [generate_spike_tensor(spikes) for spikes in notes_spikes]

white_noise_spikes = np.load(os.path.join(SPIKES_DIR, "white_noise.npy"))
tensors.append(generate_spike_tensor(white_noise_spikes))

root_tensor = generate_spike_tensor(get_spikes(root_note))

scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)
print("Scores")
print(np.round(np.array(scores)))

P_predicted = predicted_probabilities(scores)
print("P_predicted")
print(np.round(np.array(P_predicted), 2))

print("Most Cons, Least Cons, White Noise")


Scores
[0. 0. 0.]
P_predicted
[0.29 0.33 0.38]
Most Cons, Least Cons, White Noise


In [7]:
keys = ["C4", "C#4", "D4", "D#4", "E4", "F4", "F#4", "G4", "G#4", "A4", "A#4", "B4"]

for key in keys:
    print(f"{key}: {eval_simple(key)}")

C4: 81
C#4: 81
D4: 81
D#4: 81
E4: 81
F4: 81
F#4: 81
G4: 80
G#4: 81
A4: 81
A#4: 81
B4: 80


In [8]:
consonance_ordered_notes(root_note)

['C5', 'G4', 'F4', 'E4', 'D#4', 'A4', 'G#4', 'D4', 'C#4', 'B4', 'A#4', 'F#4']

In [9]:
# collect_data(kl_divergence_reverse)
# collect_data(kl_divergence)